## Kavir_RAG
Retrieval-Augmented Generation
<div style="direction: rtl; white-space: pre-wrap; line-height: 1.5;">
Retrieval = بازیابی (درست مثل سرچ کردن و پیدا کردن یک متن مرتبط از میان کلی سند)
Augmented = تقویت‌شده / غنی‌شده (یعنی اطلاعات جدیدی به چیزی اضافه شده)
Generation = تولید متن (کاری که مدل زبانی انجام می‌ده: جمله‌سازی، پاسخ‌دهی)
</div>